### BERT base_model

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset
import torch

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

In [ ]:
class NERDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = self.texts[item]
        labels = self.labels[item]

        # 텍스트 토크나이징
        encoding = self.tokenizer(text,
                                  return_tensors='pt',
                                  padding='max_length',
                                  truncation=True,
                                  max_length=self.max_len,
                                  is_split_into_words=True,
                                  return_offsets_mapping=True)
        # 오프셋 매핑을 사용해 원본 토큰과 매핑된 레이블을 연결
        labels = [labels[i] for i, offset in enumerate(encoding.offset_mapping[0]) if offset[0] == 0]

        # 레이블을 텐서로 변환
        labels += [0] * (self.max_len - len(labels))  # 패딩 토큰에 대한 레이블
        labels = torch.tensor(labels)

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels}

In [ ]:
texts = [["XXXX", "-", "Classic", "Black"]]
labels = [[0, 0, 0, 1]]  # 'Black'이 색상 엔티티라고 가정

tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
max_len = 128

# 데이터셋 인스턴스 생성
dataset = NERDataset(texts, labels, tokenizer, max_len)


In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)  # 예를 들어 '색상'과 '비색상' 2개의 레이블

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    # eval_dataset=validation_dataset,  # 평가 데이터셋이 있다면 추가
)

trainer.train()
